In [1]:
from azureml.core import Workspace, Environment, ScriptRunConfig, Experiment 
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

In [2]:
ws = Workspace.from_config()

# Creating a managed Compute target 

We may be interested to use the cloud `Compute` resources for two reasons: 
- For large data and large models local machine is not enough to store and run the computation. For this we can use `compute instances` and/or `compute clusters`. 
- It is natural to use cloud computation for model deployment. For this we use `Inference clusters`. 

Here we discuss managing compute target for the first purpose in mind. We discuss managing the compute target for the deployment later. 

We use `AmlCompute` object from `azureml-sdk` to provision the computation target. 

In [3]:
compute_name = 'rk-test-compute'

compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', 
                                                       max_nodes=2)

rk_cluster = ComputeTarget.create(ws, compute_name, compute_config)
rk_cluster.wait_for_completion(show_output=True)

SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


# Check for an existing compute target 

Here we can avoide creating additional cluster if it is already exists. For this we use `ComputeTargetException` object.  

In [4]:
try:
    rk_cluster = ComputeTarget(ws, compute_name)
    print('Found existing cluster')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', 
                                                       max_nodes=2)

    rk_cluster = ComputeTarget.create(ws, compute_name, compute_config)
    
rk_cluster.wait_for_completion(show_output=True)

Found existing cluster
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


# Use compute targets 

Now we create a vertual environment in the compute target. We can configure a python script to run on the cloud computation target with specified virtual environment using `ScriptRunConfig` object. 

In [6]:
training_env = Environment.get(workspace=ws, 
                               name='training_environment')

script_config = ScriptRunConfig(source_directory='.', 
                               script='script_1.py', 
                               environment=training_env,  # environment specified 
                               compute_target=rk_cluster  # computation target on cloud 
                               )

In [7]:
# rest of the code is similar to the previous one 
experiment = Experiment(workspace=ws, name='azureml-demo-exp')
run = experiment.submit(config=script_config)
run.wait_for_completion(show_output=True)

RunId: azureml-demo-exp_1633634766_fb3c25e7
Web View: https://ml.azure.com/runs/azureml-demo-exp_1633634766_fb3c25e7?wsid=/subscriptions/54245888-2ffe-41fa-b080-67a29997b41c/resourcegroups/rg-dataservices-sandbox-01/workspaces/ds_dev_01&tid=4ef6e02a-f252-4618-a1dc-03bd2f93157d

Streaming azureml-logs/20_image_build_log.txt

2021/10/07 19:26:13 Downloading source code...
2021/10/07 19:26:14 Finished downloading source code
2021/10/07 19:26:14 Creating Docker network: acb_default_network, driver: 'bridge'
2021/10/07 19:26:15 Successfully set up Docker network: acb_default_network
2021/10/07 19:26:15 Setting up Docker configuration...
2021/10/07 19:26:15 Successfully set up Docker configuration
2021/10/07 19:26:15 Logging in to registry: 0319839584634a4d8899cb105b49c173.azurecr.io
2021/10/07 19:26:16 Successfully logged into 0319839584634a4d8899cb105b49c173.azurecr.io
2021/10/07 19:26:16 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_networ


Streaming azureml-logs/55_azureml-execution-tvmps_d616799ab6c5e0c7f4a2fc8326de22680b55a0e5aaa1e2a74e3aecdd67c24324_d.txt

2021-10-07T19:35:23Z Running following command: /bin/bash -c sudo blobfuse /mnt/batch/tasks/shared/LS_root/jobs/ds_dev_01/azureml/azureml-demo-exp_1633634766_fb3c25e7/mounts/workspaceblobstore --tmp-path=/mnt/batch/tasks/shared/LS_root/jobs/ds_dev_01/azureml/azureml-demo-exp_1633634766_fb3c25e7/caches/workspaceblobstore -o ro --file-cache-timeout-in-seconds=1000000 --cache-size-mb=24782 -o nonempty -o allow_other --config-file=/mnt/batch/tasks/shared/LS_root/jobs/ds_dev_01/azureml/azureml-demo-exp_1633634766_fb3c25e7/configs/workspaceblobstore.cfg --log-level=LOG_WARNING
2021-10-07T19:35:23Z Successfully mounted a/an Blobfuse File System at /mnt/batch/tasks/shared/LS_root/jobs/ds_dev_01/azureml/azureml-demo-exp_1633634766_fb3c25e7/mounts/workspaceblobstore
2021-10-07T19:35:23Z The vmsize standard_ds11_v2 is not a GPU VM, skipping get GPU count by running nvidia-smi

{'runId': 'azureml-demo-exp_1633634766_fb3c25e7',
 'target': 'rk-test-compute',
 'status': 'Completed',
 'startTimeUtc': '2021-10-07T19:35:20.748548Z',
 'endTimeUtc': '2021-10-07T19:36:22.171693Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'ebde9b4e-a404-453b-9931-f7acb696aa1c',
  'azureml.git.repository_uri': 'https://github.com/roshankoirala/MLOps.git',
  'mlflow.source.git.repoURL': 'https://github.com/roshankoirala/MLOps.git',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': 'cf58f46e4a0bde9e6012ad78d77de1497aa012ce',
  'mlflow.source.git.commit': 'cf58f46e4a0bde9e6012ad78d77de1497aa012ce',
  'azureml.git.dirty': 'True',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'script_1.py',
  'command': '',
  'useAbsolutePath': False,
  'argument

# Delete compute cluster 

`Compute` resources on the cloud are one of the most expensive one. So after the use it is good idea to delete the cluster. We may also stop but it may charge some money still. 

In [8]:
rk_cluster.delete()